In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

In [7]:
GRID_SIZE = 100
EMPTY_HOUSE_RATIO = 0.1
BLUE_PARTY_RATIO = .5
RED_PARTY_RATIO = 1-BLUE_PARTY_RATIO
RELOCATION_THRESHOLD = 0.74
EMPTY = 0
BLUE = 1
RED = 2

In [8]:
flat_grid = np.empty(GRID_SIZE*GRID_SIZE)
empty_error = -1.0
blue_error = -1.0
red_error = -1.0
empty_count = 0
blue_count = 0
red_count = 0
def fill_flat_grid(flat_grid,empty_count,blue_count,red_count,empty_error,blue_error,red_error):
    for i in range(len(flat_grid)):
        empty_error = EMPTY_HOUSE_RATIO*(GRID_SIZE*GRID_SIZE) - empty_count
        blue_error = (1-EMPTY_HOUSE_RATIO)*BLUE_PARTY_RATIO*(GRID_SIZE*GRID_SIZE) - blue_count
        red_error = (1-EMPTY_HOUSE_RATIO)*RED_PARTY_RATIO*(GRID_SIZE*GRID_SIZE) - red_count
        if empty_error > blue_error and empty_error > red_error:
            flat_grid[i] = EMPTY
            empty_count+=1
        elif blue_error > empty_error and blue_error > red_error:
            flat_grid[i] = BLUE
            blue_count+=1
        elif red_error > empty_error and red_error > blue_error:
            flat_grid[i] = RED
            red_count+=1
        #Tiebreakers
        elif blue_error == red_error or blue_error == empty_error:
            flat_grid[i] = BLUE
            blue_count+=1
        elif red_error == empty_error:
            flat_grid[i] = RED
            red_count+=1
        print(f"Decision:  {flat_grid[i]}, empty_error: {empty_error}, blue_error: {blue_error}, red_error: {red_error}")
        print("\n")
fill_flat_grid(flat_grid,empty_count,blue_count,red_count,empty_error,blue_error,red_error)
print(flat_grid)

Decision:  1.0, empty_error: 1000.0, blue_error: 4500.0, red_error: 4500.0


Decision:  2.0, empty_error: 1000.0, blue_error: 4499.0, red_error: 4500.0


Decision:  1.0, empty_error: 1000.0, blue_error: 4499.0, red_error: 4499.0


Decision:  2.0, empty_error: 1000.0, blue_error: 4498.0, red_error: 4499.0


Decision:  1.0, empty_error: 1000.0, blue_error: 4498.0, red_error: 4498.0


Decision:  2.0, empty_error: 1000.0, blue_error: 4497.0, red_error: 4498.0


Decision:  1.0, empty_error: 1000.0, blue_error: 4497.0, red_error: 4497.0


Decision:  2.0, empty_error: 1000.0, blue_error: 4496.0, red_error: 4497.0


Decision:  1.0, empty_error: 1000.0, blue_error: 4496.0, red_error: 4496.0


Decision:  2.0, empty_error: 1000.0, blue_error: 4495.0, red_error: 4496.0


Decision:  1.0, empty_error: 1000.0, blue_error: 4495.0, red_error: 4495.0


Decision:  2.0, empty_error: 1000.0, blue_error: 4494.0, red_error: 4495.0


Decision:  1.0, empty_error: 1000.0, blue_error: 4494.0, red_error: 4494.0



In [9]:
grid = flat_grid.reshape(GRID_SIZE,GRID_SIZE)
print(grid)

[[1. 2. 1. ... 2. 1. 2.]
 [1. 2. 1. ... 2. 1. 2.]
 [1. 2. 1. ... 2. 1. 2.]
 ...
 [1. 2. 0. ... 2. 0. 1.]
 [2. 0. 1. ... 0. 1. 2.]
 [0. 1. 2. ... 1. 2. 0.]]


In [ ]:
def relocate(grid_in, r, c):
    #find indices of empty values
    r_indices, c_indices = np.where(grid == EMPTY)
    empty_indices = list(zip(r_indices, c_indices))
    destination = random.choice(empty_indices)
    destination_r, destination_c = destination
    grid_in[destination_r][destination_c] = grid[r, c]
    grid_in[r,c] = EMPTY

#shuffle 
shuffled = True
while shuffled == True:
    shuffled = False
    for row in range(GRID_SIZE):
        for col in range(GRID_SIZE):
            #tally neighbors
            total_neighbors = 0
            bad_neighbors = 0
            for neighbor_row in range(max(0, row-1), min(GRID_SIZE, row+2)):
                for neighbor_col in range(max(0, col-1), min(GRID_SIZE, col+2)):
                    if(grid[neighbor_row][neighbor_col] != grid[row][col] and grid[neighbor_row][neighbor_col] != EMPTY):
                        bad_neighbors+=1
                    total_neighbors+=1
            if(bad_neighbors / total_neighbors > RELOCATION_THRESHOLD):
                relocate(grid, row, col)
                shuffled = True



print(grid)




KeyboardInterrupt: 